In [4]:
from openai import OpenAI

# python 实例化 不需要new对象，，没有{}
client = OpenAI(
    api_key='sk-676db137e5804e5db22375c19a69d580',
    base_url='https://api.deepseek.com/v1'
)

In [5]:
# 轻量级数据库
import sqlite3

In [6]:
# 打开链接
conn=sqlite3.connect("test3.db")
# 数据库操作的句柄
cursor=conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS employees(
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary INTEGER
)
""")

In [7]:
# list 
sample_data = [ 
    # 元祖类型 有顺序的  tuple 
    (6, "黄佳", "销售", 50000), 
    (7, "宁宁", "工程", 75000), 
    (8, "谦谦", "销售", 60000), 
    (9, "悦悦", "工程", 80000), 
    (10, "黄仁勋", "市场", 55000),
    (11, "杜经理", "工程", 80000)
]
# 插入sql
cursor.executemany(
    "INSERT INTO employees VALUES(?,?,?,?)",
    sample_data
)
# 提交sql事务
conn.commit()

IntegrityError: UNIQUE constraint failed: employees.id

In [8]:
# text2sql 给llm什么？ context
# llm会sql gemini 博士级别
# 表结构？ 作为上下文给prompt schema
schema = cursor.execute("PRAGMA table_info(employees)").fetchall()
print(schema)
# 列表推导式
schema_str = "CREATE TABLE EMPLOYEES (\n" + "\n".join([f"{col[1]} {col[2]}" for col in schema]) + "\n)"
print(schema_str)

[(0, 'id', 'INTEGER', 0, None, 1), (1, 'name', 'TEXT', 0, None, 0), (2, 'department', 'TEXT', 0, None, 0), (3, 'salary', 'INTEGER', 0, None, 0)]
CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)


In [9]:
def ask_deepseek(query,schema):
    prompt=f"""
    这是一个数据库的Schema:
    {schema}
    根据这个schema,请输出一个SQL查询，来回答以下问题：
    只输出SQL查询语句本身，不要使用任何markdown格式，不要包含反引号、代码块标记或额外说明。
    问题：{query}
    """
    
    print(prompt)
    response=client.chat.completions.create(
        model="deepseek-chat",
        max_tokens=2048,
        messages=[{
            "role":"user",
            "content":prompt
        }],
        temperature=0
    )
    return response.choices[0].message.content

In [10]:
question="工程部门员工的姓名和工资是多少"
sql_query=ask_deepseek(question,schema_str)
print(sql_query)


    这是一个数据库的Schema:
    CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)
    根据这个schema,请输出一个SQL查询，来回答以下问题：
    只输出SQL查询语句本身，不要使用任何markdown格式，不要包含反引号、代码块标记或额外说明。
    问题：工程部门员工的姓名和工资是多少
    
SELECT name, salary FROM EMPLOYEES WHERE department = '工程';


In [11]:
# ai 生成sql 能力潜入开发流程中
results=cursor.execute(sql_query).fetchall()
print(results)

[('宁宁', 75000), ('悦悦', 80000), ('杜经理', 80000)]


In [12]:
question="在销售部门增加一个新员工，姓名为张三，工资为45000"
sql_question=ask_deepseek(question,schema_str)
print(sql_question)


    这是一个数据库的Schema:
    CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)
    根据这个schema,请输出一个SQL查询，来回答以下问题：
    只输出SQL查询语句本身，不要使用任何markdown格式，不要包含反引号、代码块标记或额外说明。
    问题：在销售部门增加一个新员工，姓名为张三，工资为45000
    
INSERT INTO EMPLOYEES (name, department, salary) VALUES ('张三', '销售', 45000);


In [13]:
cursor.execute(sql_query)
conn.commit()

In [14]:
# llm加持的后台管理系统 
# 数据库平权，小编，业务，任何人加入后台运营和维护

In [15]:
question='删除市场部门的黄仁勋'
sql_query=ask_deepseek(question,schema_str)
print(sql_query)
cursor.execute(sql_query)
conn.commit


    这是一个数据库的Schema:
    CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)
    根据这个schema,请输出一个SQL查询，来回答以下问题：
    只输出SQL查询语句本身，不要使用任何markdown格式，不要包含反引号、代码块标记或额外说明。
    问题：删除市场部门的黄仁勋
    
DELETE FROM EMPLOYEES WHERE department = '市场' AND name = '黄仁勋';


<function Connection.commit()>

In [19]:
question="查询所有员工的信息并展示"
sql_query=ask_deepseek(question,schema_str)
print(sql_query)
results=cursor.execute(sql_query).fetchall()


    这是一个数据库的Schema:
    CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)
    根据这个schema,请输出一个SQL查询，来回答以下问题：
    只输出SQL查询语句本身，不要使用任何markdown格式，不要包含反引号、代码块标记或额外说明。
    问题：查询所有员工的信息并展示
    
SELECT * FROM EMPLOYEES;
